## 1. Idiomatic RobotFramework

![RobotFramework](/files/img/robotframework.png "RobotFramework")

### TA是什么？

#### TA是软件开发的一种

### 什么样的场景适合/不适合TA？

### 在RobotFramework里面，suite是什么？

* Documentation
* Meta Data
* Force Tags
* Suite Setup和Suite Teardown
* 所有case的默认Test Setup, Test Teardown和Test Timeout

### 有哪些suite组织上的常见问题？

* 只把文件当作suite
* 一个suite仅包含一个test
* 一个suite包含超过10个test
* setup和teardown的步骤都在test中
* suite的目录没有业务逻辑性(如: 根据team或者部门来组织suite目录结构)

### 什么时候需要Variable File？

* 获取动态数据
* 复杂的数据结构

In [ ]:
# sample test_var.py
import socket
import fcntl
import struct

def get_variables(iface):
    return {'LOCAL_IP': _get_ip_address(iface)}

def _get_ip_address(ifname):
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    return socket.inet_ntoa(fcntl.ioctl(
        s.fileno(),
        0x8915,  # SIOCGIFADDR
        struct.pack('256s', ifname[:15])
    )[20:24])

### 什么时候需要用到Listener?

* 测试之外的需求, 如统计, 调试等
* 统一要求的测试需求, 如文件清理, CI环境恢复等
* 临时的测试需求

In [ ]:
# sample test_listener.py
import os
import time

class RunningInspector:
    ROBOT_LISTENER_API_VERSION = 2

    def __init__(self, output_file='inspector.report'):
        self._output_file = os.path.abspath(output_file)
        # {'log': [10, 20, 30], 'quit': [300]}
        self._report = {}
        self._duration_stack = []

    def start_keyword(self, name, attrs):
        self._duration_stack.append(time.time())

    def end_keyword(self, name, attrs):
        duration = time.time() - self._duration_stack.pop()
        self._report.setdefault(name, []).append(duration)

    def close(self):
        ''' write report file '''
        with open(self._output_file, 'w+b') as f:
            for kw_name, durations in self._report.iteritems():
                f.write(kw_name + '\n')
                f.write('\tcount: %d, duration: %d\n' % (len(durations), sum(durations)))

#### [一个复杂的示例: rdb](http://gitlab.china.nsn-net.net/ta/rdb)

### 给Keyword提供参数有哪些方法？

#### 方法一：通过[Arguments]

```
Send data to Server should return expect data back
    [Arguments]    ${data}    ${expect data}
    EchoClient.send    ${data}
    ${received data}    EchoClient.read
    Should Be Equal    ${received data}    ${data}
```

#### 方法二：嵌入Keyword名字中

```
Send "${data}" to Server should return "${expect data}" back
    EchoClient.send    ${data}
    ${received data}    EchoClient.read
    Should Be Equal    ${received data}    ${data}
```

### 怎么写一个library？

In [ ]:
# sample echo_lib.py
import socket

def connect_to_echo_client(host='127.0.0.1', port=50007, timeout=0.5):
    _sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    _sock.settimeout(timeout)
    _sock.connect(host, int(port))
    return _sock

def disconnect_from_echo_client(_sock):
    _sock.close()

In [ ]:
# sample echo2_lib.py
import socket

class EchoClient(object):
    ROBOT_LIBRARY_SCOPE = 'GLOBAL'
    def __init__(self, host='127.0.0.1', port=50007, timeout=0.5):
        self._host = host
        self._port = int(port)
        self._timeout = float(timeout)
        self._sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self._sock.settimeout(self._timeout)
        self._buffer = 1024

    def connect(self):
        self._sock.connect((self._host, self._port))

    def disconnect(self):
        self._sock.close()

#### 这两种library区别在哪里？